In [ ]:
import nltk
# tokenizing in various ways
from nltk.tokenize import line_tokenize, sent_tokenize, WordPunctTokenizer, word_tokenize
# stopwwords collection
from nltk.corpus import stopwords
# stemming
from nltk.stem import PorterStemmer
# to save and load models

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import pickle

# regular expressions
import re
# lemmatize like stem
from nltk.stem import WordNetLemmatizer
# to ramdomly shuffle
import random
# collection of dictionary, lemma ,examples. 
from nltk.corpus import wordnet 
# To Wrap up the sklearn classifiers to be used in NLP for classifying
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.metrics import *
from sklearn.naive_bayes import MultinomialNB

import numpy as np
import pandas as pd

In [ ]:
lm = WordNetLemmatizer()

In [ ]:
pos_file = open('Data/positive.txt','r').read()
neg_file = open('Data/negetive.txt','r').read()

In [ ]:
print(len(pos_file),len(neg_file))

In [ ]:
# Removes stopwords and storing only storing alpabets and numbers and imp symbols also lemamtizing to decrease number of features
class Preprocessor:

    def preprocessor(self,doc):
        new_doc = ' '.join([lm.lemmatize(word) for word in word_tokenize(doc) if word not in stopwords.words('english') and (word.isalpha() or word.isnumeric() or (word in ['!','.',',']))])
        return new_doc


In [ ]:
pre_process = Preprocessor()

In [ ]:
pos_file_new = pre_process.preprocessor(pos_file)
neg_file_new = pre_process.preprocessor(neg_file)

In [ ]:
pos_file=None
neg_file=None

In [ ]:
print(len(pos_file_new),len(neg_file_new))

In [ ]:
# storing preprocessor object
file = open('model_pickle/Count_vectorizer/preprocessor.pkl','wb')
pickle.dump(pre_process,file)
file.close()

In [ ]:
doc = []
for sentence in sent_tokenize(pos_file_new):
    doc.append((sentence,'pos'))
for sentence in sent_tokenize(neg_file_new):
    doc.append((sentence,'neg'))

In [ ]:
random.shuffle(doc)

In [ ]:
class by_count_vectorizer:

    def __init__(self,doc):
        self.doc = doc
            
    def feature_making(self):
        X_train=[]
        y_train=[]
        for sent,cat in self.doc:
            X_train.append(sent)
            y_train.append(cat)
        return X_train,y_train
    
    def convert_y_int(self,y_train,var):
        for row in range(0,len(y_train)):
            if y_train[row]==var:
                 y_train[row]= 0
            else:
                y_train[row] = 1
        return y_train
    
    def plotter(self,cross_val,classifier):
                     
        xar =[]
        yar= []
        
        x=y=0
        for rows in cross_val:
            x += 1
            print(x)
            if classifier.predict(rows)==1:
                y+=1
            else:
                y-=1
            xar.append(x)
            yar.append(y)
                    
            ax1.clear()
            ax1.plot(xar,yar,color='green')
            plt.show()
            time.sleep(100)
#             animation.FuncAnimation(fig,self.plotter,interval=1000)
    
    def check_result(self,feature,classifier):
        if(classifier.predict(feature)==1):
            print('positive')
        else:
            print('negetive')
    
    def classify_demo(self):
        cv = CountVectorizer()
        X_train,y_train = self.feature_making()
        y_train = self.convert_y_int(y_train,'neg')
        X = cv.fit_transform(X_train)
        print(X.toarray())
#         print(X_train)
        x_train,x_test,y_train,y_test = train_test_split(X,y_train,test_size=0.1)
        x_train,x_cross,y_train,y_cross = train_test_split(x_train,y_train,test_size=0.2)
        file=open('Data/X_test_cv_data.pkl','wb')
        pickle.dump(x_test,file)
        file.close()
        file=open('Data/Y_test_cv_data.pkl','wb')
        pickle.dump(y_test,file)
        file.close()
        
        mnb = MultinomialNB()
        mnb.fit(x_train,y_train)
        y_pred = mnb.predict(x_cross)
        print("Accuracy by f1 score ", f1_score(y_cross,y_pred)*100)
        if(f1_score(y_cross,y_pred)>0.60):
            file = open('model_pickle/Count_vectorizer/count_vect.pkl','wb')
            pickle.dump(cv,file)
            file.close()
            file = open('model_pickle/Count_vectorizer/classifier.pkl','wb')
            pickle.dump(mnb,file)
            file.close()
        feature = cv.transform(["The movie was pleasant"])
        self.check_result(feature,mnb)

In [ ]:
cv_classifier = by_count_vectorizer(doc)

In [ ]:
cv_classifier.classify_demo()

In [ ]:
class by_tfid_vectorizer:

    def __init__(self,doc):
        self.doc = doc
    
    def feature_making(self):
        X_train=[]
        y_train=[]
        for sent,cat in self.doc:
            X_train.append(sent)
            y_train.append(cat)
        return X_train,y_train
    
    def convert_y_int(self,y_train,var):
        for row in range(0,len(y_train)):
            if y_train[row]==var:
                 y_train[row]= 0
            else:
                y_train[row] = 1
        return y_train
    
    def check_result(self,feature,classifier):
        if(classifier.predict(feature)==1):
            print('positive')
        else:
            print('negetive')
    
    def classify_demo(self):
        cv = TfidfVectorizer(min_df=1,stop_words='english')
        X_train,y_train = self.feature_making()
        y_train = self.convert_y_int(y_train,'neg')
        X = cv.fit_transform(X_train)
        x_train,x_test,y_train,y_test = train_test_split(X,y_train,test_size=0.1)
        x_train,x_cross,y_train,y_cross = train_test_split(x_train,y_train,test_size=0.2)
        file=open('Data/X_test_tfidf_data.pkl','wb')
        pickle.dump(x_test,file)
        file.close()
        file=open('Data/Y_test_tfidf_data.pkl','wb')
        pickle.dump(y_test,file)
        mnb = MultinomialNB()
        mnb.fit(x_train,y_train)
        y_pred = mnb.predict(x_cross)
        print("Accuracy by f1 score {} and by accuracy_score is {}".format(f1_score(y_cross,y_pred)*100,accuracy_score(y_cross,y_pred)*100))
        if(f1_score(y_cross,y_pred)>0.60):
            feature_make_file = open('model_pickle/TFidf_vectorizer/tfid_vectorizer.pkl','wb')
            pickle.dump(cv,feature_make_file)
            feature_make_file.close()
            model_saver = open('model_pickle/TFidf_vectorizer/classifier.pkl','wb')
            pickle.dump(mnb,model_saver)
            model_saver.close()
        feature = cv.transform(["The movie was not good and I hated it"])
        
        self.check_result(feature,mnb)
        

In [ ]:
tf_classify = by_tfid_vectorizer(doc)

In [ ]:
tf_classify.classify_demo()